# Shipping land route generation

In [14]:
# Allow access to Python imports from higher level folder
import sys
sys.path.append('..')

# Define directory locations
TEST_DIR = '../test_data/usa/'
IN_SHAPE_DIR = '../test_data/usa/in_shape/'

# ALLFED libs
from features.io import write_features, load_features
from geometry.project import project_features_to_world, project_features_from_world
from geometry.snap import snap_features
from geometry.merge import merge_features
from geometry.line import split_features_by_distance

In [4]:
# Load road and rail shapefiles. Note that geometries have 
# already been simplified using Douglas Peucker to a 0.01 degree tolerance
ROAD_PATH = TEST_DIR + 'global_roads_v1_simplified_001_test_usa.shp'
RAIL_PATH = TEST_DIR + 'ne_10m_railroads_test_usa.shp'

road_features = load_features(ROAD_PATH, {'type': 'road'})
rail_features = load_features(RAIL_PATH, {'type': 'rail'})

In [5]:
# Project features
project_features_to_world(road_features)
project_features_to_world(rail_features)

In [7]:
# Merge contiguous sections
road_features = merge_features(road_features)
rail_features = merge_features(rail_features)

In [10]:
# Combine road and rail and split out network every 10km to provide 
# regularly spaced attachment points for our eventual network
route_features = road_features + rail_features
route_features = split_features_by_distance(route_features, 10000)

In [11]:
# Snap lines to each other within a threshold
route_features = snap_features(5000, route_features) # 5km threshold

In [12]:
# Add length information
for rf in route_features:
    rf.update_length('km')

In [15]:
# Write output to shapefile
project_features_from_world(route_features)
write_features(route_features, IN_SHAPE_DIR + 'routes.shp')